# Extracting data from the data engineering networking WhatsApp group

In [31]:
import pandas as pd
import re
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter

# Taking a txt file and transforming it into a dataframe

In [32]:
path = "D:/engenharia de dados/files/whats/networking.txt"


with open(path, "r", encoding="utf-8") as f:
    content = f.readlines()

default = r"^(?:\d{1,2}\/\d{1,2}\/\d{4}) (\d{2}:\d{2}) - (?:(.*?): )?(.*)$"

data = []
current_message = None

for line in content:
    match = re.match(default, line)
    if match:
        hour, sender, message = match.groups()
        current_message = [hour, sender if sender else "SISTEMA", message]
        data.append(current_message)
    else:
        if current_message:
            current_message[2] += "\n" + line 

df = pd.DataFrame(data, columns=["Hora", "Remetente", "Mensagem"])

print(df[["Hora", "Remetente"]])


      Hora Remetente
0    10:09   SISTEMA
1    19:08   SISTEMA
2    19:08   SISTEMA
3    19:08   SISTEMA
4    10:09   SISTEMA
..     ...       ...
669  17:24   SISTEMA
670  17:36   SISTEMA
671  17:40   SISTEMA
672  17:52   SISTEMA
673  18:03   SISTEMA

[674 rows x 2 columns]


# Filtering only messages that contain LinkedIn and transforming the data only into LinkedIn links

In [33]:
df_linkedin = df[df["Mensagem"].str.contains(r"linkedin\.com", case=False, na=False)].copy()

df_linkedin["Mensagem"] = df_linkedin["Mensagem"].str.extract(
    r"((?:https?://)?(?:www\.)?linkedin\.com[^\s]+)",
    expand=False
)

df_linkedin["Mensagem"] = df_linkedin["Mensagem"].apply(
    lambda x: "https://" + x if pd.notnull(x) and not x.startswith("http") else x
)

df_linkedin.rename(columns={"Mensagem": "Linkedin"}, inplace=True)

print(df_linkedin[["Hora", "Linkedin"]])


      Hora                                           Linkedin
168  12:00        https://www.linkedin.com/in/iasmim-horrana/
169  12:00            https://www.linkedin.com/in/max-mitsuya
170  12:00  https://www.linkedin.com/in/jo%C3%A3o-victor-1...
176  12:01  https://www.linkedin.com/in/maria-eduarda-nasc...
178  12:01  https://www.linkedin.com/in/ygor-amaro-114613231/
..     ...                                                ...
611  14:04  https://www.linkedin.com/in/israel-chaves-a321...
613  14:04  https://www.linkedin.com/in/valter-perez-50283...
614  14:05  https://www.linkedin.com/in/lethicia-lima-0904...
616  14:06   https://www.linkedin.com/in/kaua-silva-b37b45259
617  14:06         https://www.linkedin.com/in/marcos-soares/

[246 rows x 2 columns]


# Generating the Excel file, adjusting the column widths, and turning the LinkedIn column into a hyperlink

In [34]:
path_excel = r"D:\\engenharia de dados\\files\\whats\\networking.xlsx"

df_linkedin.to_excel(path_excel, index=False)

wb = load_workbook(path_excel)
ws = wb.active

for row in range(2, ws.max_row + 1):
    cell = ws[f"C{row}"]
    url = cell.value
    if url:
        cell.hyperlink = url
        cell.value = "Linkedin"
        cell.style = "Hyperlink"

for col_cells in ws.columns:
    max_len = 0
    col_letter = get_column_letter(col_cells[0].column)
    for c in col_cells:
        if c.value is not None:
            max_len = max(max_len, len(str(c.value)))
    ws.column_dimensions[col_letter].width = max(10, max_len + 2)

wb.save(path_excel)
